In [ ]:
import os
import sys

sys.path.insert(0, os.path.abspath("../utils"))
from aoc_utils import load_data, check

In [ ]:
import re

In [ ]:
data = load_data(2024, 13)

In [ ]:
# data, part_1, part_2
tests = [
    (
        """Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279
""",
        480,
        875318608908,
    ),
]

# Part 1

In [ ]:
def count_tokens(data, offset=0, ub=100):
    tokens = 0
    for machine_desc in data.split("\n\n"):
        ax, ay, bx, by, px, py = (int(v) for v in re.findall(r"(\d+)", machine_desc))
        px += offset
        py += offset
        if ax * by != ay * bx:
            # the (real) solution is unique
            if not (ax and ay and bx and by):
                raise NotImplementedError("Zero coefficients are not supported.")
            b, r = divmod(ax * py - ay * px, ax * by - ay * bx)
            if r != 0:
                # b is not an integer
                continue
            a, r = divmod(px - b * bx, ax)
            if r == 0 and a >= 0 and b >= 0 and (
                ub is None
                or (a <= ub and b <= ub)
            ):
                tokens += 3 * a + b
        else:
            raise NotImplementedError("Non unique solutions are not supported.")
    return tokens

In [ ]:
check(count_tokens, tests)
count_tokens(data)

# Part 2

In [ ]:
check(count_tokens, tests, 2, offset=10000000000000, ub=None)
count_tokens(data, offset=10000000000000, ub=None)

# Additional test cases

Zero coefficients were (supposedly) not too difficult to handle, but the resulting code was a bit of a mess and not thoroughly tested.  
I only kept the part handling linearly dependent equations.

In [ ]:
more_tests = [
    (
        # Infinite solutions (only B is pressed)
        """Button A: X+3, Y+1
Button B: X+6, Y+2
Prize: X=24, Y=8
""",
        4,
        None,
    ),
    (
        # Infinite solutions (A and B are pressed)
        """Button A: X+5, Y+5
Button B: X+2, Y+2
Prize: X=9, Y=9
""",
        5,
        None,
    ),
    (
        # Infinite solutions (only A is pressed)
        """Button A: X+16, Y+8
Button B: X+4 Y+2
Prize: X=32, Y=16
""",
        6,
        None,
    ),
    (
        # More than a hundred presses
        """Button A: X+1, Y+1
Button B: X+3, Y+5
Prize: X=601, Y=1001
""",
        0,
        None,
    ),
    (
        # Negative solution: B - 2*A
        """Button A: X+1, Y+1
Button B: X+5, Y+6
Prize: X=3, Y=4
""",
        0,
        None,
    ),
    (
        # Infinite solutions (A and B are pressed)
        """Button A: X+10, Y+5
Button B: X+4, Y+2
Prize: X=18, Y=9
""",
        5,
        None,
    ),
    (
        # No solutions
        """Button A: X+10, Y+5
Button B: X+4, Y+2
Prize: X=17, Y=9
""",
        0,
        None,
    ),
    (
        # B presses are capped out
        """Button A: X+2, Y+2
Button B: X+1, Y+1
Prize: X=149, Y=149
""",
        174,
        None,
    ),
    (
        # A presses are capped out
        """Button A: X+4, Y+4
Button B: X+1, Y+1
Prize: X=450, Y=450
""",
        350,
        None,
    ),
    (
        # Equivalent buttons costs
        """Button A: X+3, Y+6
Button B: X+1, Y+2
Prize: X=7, Y=14
""",
        7,
        None,
    ),
    (
        # optimal is 19 A and 102 B -> 68 A and 65 B with the upper bound
        """Button A: X+37, Y+111
Button B: X+49, Y+147
Prize: X=5701, Y=17103
""",
        269,
        None,
    ),
    (
        # both buttons are (neraly) capped out
        """Button A: X+3, Y+3
Button B: X+2, Y+2
Prize: X=500, Y=500
""",
        400,
        None,
    ),
    (
        # both buttons are capped out
        """Button A: X+3, Y+3
Button B: X+2, Y+2
Prize: X=502, Y=502
""",
        0,
        None,
    ),
]

In [ ]:
import math

In [ ]:
def egcd(a, b):
    """Extended Euclidean algorithm.

    Parameters
    ----------
    a, b : int
        The input pair of integers

    Returns
    -------
    r, u, v : int
        A solution such that r = gcd(a, b) and r = u * a + v * b
    """
    s0, s1, t0, t1 = 1, 0, 0, 1
    while b:
        q, r = divmod(a, b)
        a, b = b, r
        s0, s1 = s1, s0 - q * s1
        t0, t1 = t1, t0 - q * t1
    return a, s0, t0

In [ ]:
def solve(equations):
    """Solve a pair of equations.

    Find integers a and b such that:
    - ax * a + bx * b = px
    - ay * a + by * b = py
    - a >= 0
    - b >= 0
    - the solution minimizes 3 * a + b

    Provided solutions have no upper bounds.

    Parameters
    ----------
    equations : (int, int, int), (int, int, int)
        The (ax, bx, px), (ay, by, py) strictly positive coefficients.

    Returns
    -------
    solvable : bool
        Whether there is a valid solution.
    a, b : int, int
        The solutions to the equations.
    """
    (ax, bx, px), (ay, by, py) = equations
    if ax * by != ay * bx:
        # independant equations: the (real) solution is unique
        if ax == 0:
            b, r = divmod(px, bx)
        else:
            b, r = divmod(ax * py - ay * px, ax * by - ay * bx)
        if r != 0:
            return False, (None, None)
        a, r = divmod(py - by * b, ay)
        if r == 0 and a >= 0 and b >= 0:
            return True, (a, b)
        return False, (None, None)

    # else: dependant equations
    # subcase 1: no solutions
    if ax * py != ay * px:
        return False, (None, None)

    # subcase 2: infinite solutions
    gcd, u, v = egcd(ax, bx)
    ax //= gcd
    bx //= gcd
    px //= gcd

    if ax >= 3 * bx:
        # prefer A: px * v - ax * k >= 0
        k = px * v // ax
    else:
        # prefer B: px * u + bx * k >= 0
        k = (- px * u - 1) // bx + 1
    a = px * u + bx * k
    b = px * v - ax * k
    return True, (a, b)

In [ ]:
def count_extended_tokens(data, offset=0, ub=100):
    tokens = 0
    for machine_desc in data.split("\n\n"):
        ax, ay, bx, by, px, py = (int(v) for v in re.findall(r"(\d+)", machine_desc))
        if not (ax and ay and bx and by):
            raise NotImplementedError("Zero coefficients are not supported.")
        solvable, (a, b) = solve([
            [ax, bx, px + offset],
            [ay, by, py + offset],
        ])
        if solvable:
            if ub is not None and (a > ub or b > ub):
                if a > ub:
                    k = (a - ub - 1) * math.gcd(ax, bx) // bx + 1
                    a -= k * bx // math.gcd(ax, bx)
                    b += k * ax // math.gcd(ax, bx)
                if b > ub:
                    k = (b - ub - 1) * math.gcd(ax, bx) // ax + 1
                    b -= k * ax // math.gcd(ax, bx)
                    a += k * bx // math.gcd(ax, bx)
            if ub is None or (a <= ub and b <= ub):
                tokens += 3 * a + b
    return tokens

In [ ]:
check(count_extended_tokens, tests)
check(count_extended_tokens, more_tests)